#### Общие правила
- Сделанная домашняя работа - jupyter-тетрадка.
- Пожалуйста, присваивайте блокам кода номера и/или заголовки, соответствующие заданиям. Код должен содержать комментарии, ход мысли и решения должен быть понятен из вашей тетрадки
- Максимальный балл за домашнюю работу - 20. Вы не обязаны выполнять все пункты, чтобы набрать максимальное количество баллов
- Можно использовать чужой код, но обязательно ставьте ссылки на источник
- **Deadline** - 25.04.2019

## AGRR: Automatic Gapping Resolution for Russian

#### Краткий ликбез
Примеры гэппинга в русском
 - _Ей он рассказывает одно, а нам — совершенно другое_  
 - _Кто любит арбуз, а кто — свиной хрящик_  
 - _Дайте мне две пятерки, а я вам десятку_
 - _Тогда я принял ее за итальянку, а его за шведа_
 

train + dev включают в себя больше 20 тысяч примеров, примерно треть из них иллюстрирует явление гэппинга в русском.  <br>
Гэппинг - явление, при котором в одной из сочиненных (иногда подчиненных) клауз, опускается повторяющийся предикат, но остаются выраженными его участники.
 - _Тогда я принял ее за итальянку, а его ~~принял~~ за шведа_


Терминология для описания элементов гэппинга разнится, мы используем следующую:

При разметке предложения мы выделяем следующие элементы (терминология разнится в разных источниках, бояться не нужно):
- ремнанты **R1** и **R2** - участники эллиптированного предиката (в нашем примере _его_ и _за шведа_)
- корреляты ремнантов  **cR1** и **cR2** - соответсвующие группы в полной клаузе (в нашем примере _ее_ и _за итальянку_)
- позиция опущенного (эллиптированного) предиката  **V**
- вершина конролирующего предиката **cV** (в нашем примере _принял_ из первой клаузы)

Посмотрите на пример со скобочной разметкой:

 - _Тогда я  **cV[** принял **cV]**  **cR1[** ее **cR1]**  **cR2[** за итальянку **cR2]**, а  **R1[** его **R1]**   **V[]**  **cR2[** за шведа **cR2]**._

Есть много теоретических работ, посвященных этому виду эллипсиса (прикладных решений практически нет). Желающим: 

- Лингвистическая сторона дела:
[Ross 1970](https://babel.ucsc.edu/~hank/Ross_Gapping.pdf) 
[Jackendoff 1971](https://babel.ucsc.edu/~hank/jackendoff_related_rules.pdf) 
[Steedman 1989](https://repository.upenn.edu/cgi/viewcontent.cgi?article=1861&context=cis_reports) 
[Coppock 2001](https://www.linguistics.northwestern.edu/documents/award-winners/linguistics-undergraduate-award-past-winner-coppock.pdf)
[Merchant 2016](http://home.uchicago.edu/merchant/pubs/ellipsis.revised.pdf) 

- Подходы к решениям:
[Bogdanov  2012](http://www.dialog-21.ru/media/1316/89.pdf) 
[Droganova 2018](http://www.aclweb.org/anthology/W18-6006) 
[Schuster 2018](https://arxiv.org/pdf/1804.06922.pdf) 



**NB**: Данные собраны, размечены и подготовлены к обучению, так что если вас пугает, что вы не очень понимаете явление, с которым придется работать, это не так критично, не пугайтесь. <br>
Задачу можно воспринимать как классификацию последовательности, в этом смысле она не отличается от задачи NER или POS-tagging. 

 
### Формат данных
Скобочная запись удобна для оценки разметки глазами. Сама разметка хранится в следующем виде:
csv файл, колонки разделены табуляцией.
Первая колонка - сам текст. Вторая колонка - 0 или 1 (отсутсвие или наличие гэппинга в предложении).
Следующие 6 колонок содержат оффсеты подстрок, соответсвующих элементам гэппинга(**cV, cR1, cR2, V, R1, R2**). Ячейка может быть пустой (не все элементы обязательны), может содержать как одну пару оффсетов (разделены через двоеточие), так и несколько пар (разделены через пробел, порядок пар не важен).
Посмотрите на пример:
 
**Input**  
>Аналогичным образом, среднегодовой прирост ВВП на душу населения, который в странах, расположенных к югу от Сахары, составлял в период с 1965 по 1973 год 3 процента, упал с 1980 до 1986 года на 2,8 процента, в 1987 году - на 4,4 процента и в 1989 году - на 0,5 процента.

**Output**  

| class | cV  | cR1  | cR2 | V | R1  | R2  |
| :---  | :---  | :---  | :---  | :---  | :---  | :---  |
| 1| 166:170 | 171:190 | 191:206  | 222:222 254:254  | 208:219 240:251 | 222:237 254:269 |


Такая разметка соответствует следующей скобочной записи:

 - _Аналогичным образом, среднегодовой прирост ВВП на душу населения, который в странах, расположенных к югу от Сахары, составлял в период с 1965 по 1973 год 3 процента,  **cV[** упал **cV]**  **cR1[** с 1980 до 1986 года **cR1]**  **cR2[** на 2,8 процента **cR2]**,   **R1[** в 1987 году **R1]** — **V[]** **R2[** на 4,4 процента **R2]** и  **R1[** в 1989 году **R1]** —  **V[]** **R2[** на 0,5 процента **R2]**._

## Описание заданий

#### 1. Подготовка данных (2 балла)
Скачайте данные (train, dev, test). Для работы с таблицей используйте pandas. 
Используйте параметр quoting - иначе pandas вольно обращается с кавычками в тексте и оффсеты могут съехать
> import pandas as pd <br>
import csv <br>
> train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE) 

Посчитайте, сколько в каждом из файлов примеров из класса 0 и из класса 1.


#### 2. Анализ разметки (2 балла)
Стандартный пример содержит по паре оффсетов для каждой метки **cV, cR1, cR2, V, R1, R2** , но есть и другие варианты. Посчитайте сколько их и какие случаи им соответствуют.
Представьте результаты в понятном виде, например в таблице.

#### 3. Конвертер разметки-1 (1 балл)
Напишите функцию, переводящую разметку по оффсетам в IOB/IO/BIOS(на выбор) формат и обратно. Она пригодится вам при формировании входа для сети для решения классификации последовательности.

Напоминание про IOB(ES) разметку:
Каждому элементу последовательности (например токену или символу) сопоставляется одна из типов меток: O - outside, B - begin,
I - inside, E - end, s - single, в зависимости от того, входит ли этот элемент в интересующую нас подстроку. Например, в задаче NER метка B-LOC означает, что данный элемент - первый в подстроке, соответствующей названию локации. 

Пример подходящей для нашей задачи IOB-разметки (на уровне токенов):

| Тогда | я   | принял  | ее    | за    | итальянку  | а   | его  | за   | шведа |
| :---  | :---| :---    | :---  | :---  | :---       | :---| :--- | :--- | :---  |
| O     | O   | B-cV    | B-cR1 | B-cR2 | I-cR2      | O   | B-R1 | B-R2 | I-R2  |


#### 4. Конвертер разметки-2 (1 балл)
Напишите функцию, переводящую разметку по оффсетам в скобочную запись. Она пригодится вам при анализе получаемых результатов.

#### 5. Бинарная классификация (6 баллов (+2))
Постройте простую полносвязную сеть, берущую на вход все предложение и на выходе предсказывающую наличие или отсутствие гэппинга в предложении. Признаки: использование эмбеддингов +1 балл, использование pos-тэгов +1 балл

#### 6. Оценка результатов бинарной классификации (1 балл)
Обученной сеткой разметьте тестовые данные, с помощью готового скрипта agrr_metrics.py оцените полученный результат.
Пример вызова из командной строки:
> python3 agrr_metrics.py -b test.csv output.csv


#### 7. Классификация последовательности (7 баллов) 
Решите задачу полной разметки как классификацию последовательности. Напишите RNN, которая берет на вход последовательность символов длины n, выдает n IOB/IO/BIOS меток. [реализация NER от Stanford](https://cs230-stanford.github.io/pytorch-nlp.html)

#### 8. Оценка результатов полной разметки (1 балл)
Обученной сеткой разметьте тестовые данные, с помощью готового скрипта agrr_metrics.py оцените полученный результат.
Пример вызова из командной строки:
> python3 agrr_metrics.py test.csv output.csv

#### 9. Анализ результатов (до 4 баллов)
Проведите анализ ошибок. Посмотрите на fn, fp примеры, приведите несколько таких предложений в скобочной записи (если вы решали задачу классификации последовательности). Можно ли попытаться обобщить получаемые ошибки?